In [ ]:
using NBInclude
@nbinclude("randommap.ipynb")

In [ ]:
function test_random_map(n::Int, x::Int, num_maps::Int)
    println(n, " gene(s), ", x, " duplicate(s)\n")
    src, target = generate_genomes_with_xdup(n, x)
    dcj_dist_list, P_dedup, min_dist, min_src, min_map, max_dist, max_src, max_map = randommap(src, target, num_maps, "none")
end 


In [ ]:
n = 3
x = 1
num_maps = 10

test_random_map(n, x, num_maps)

dcj_dist_list, target, min_src, min_map, max_src, max_map


### visualization 

max_value = length(target)
println("\n**\nmax dcj distance = ", max_value, "\n")

# SOCS of distribution
# shape -- skewed right usually 
# outliers - none 
# center - depends 
# spread -- narrow 
min = minimum(dcj_dist_list)
max = maximum(dcj_dist_list)
avg = mean(dcj_dist_list)

println("min=", min)
println("max=", max)
println("avg=", avg, "\n")
println("range=", max-min)


# range for x-ticks from 0 to max_value with a step of 2
xticks = 0:1:max_value

# Create a histogram
histogram(dcj_dist_list, title="Distribution", label="", xlabel="DCJ Distance", ylabel="Frequency",  xlims=(0, max_value+2), xticks=(xticks, xticks))



In [ ]:
n = 3
x = 3
println(n, " gene(s), ", x, " duplicate(s)")
src, target = generate_genomes_with_xdup(n, x)

num_maps = 10

dcj_dist_list = randommap(src, target, num_maps, "none")

### visualization 

max_value = length(target)+2
println("\n**\nmax dcj distance = ", max_value, "\n")

# SOCS of distribution
# shape -- skewed right usually 
# outliers - none 
# center - depends 
# spread -- narrow 
min = minimum(dcj_dist_list)
max = maximum(dcj_dist_list)
avg = mean(dcj_dist_list)

println("min=", min)
println("max=", max)
println("avg=", avg, "\n")
println("range=", max-min)


# range for x-ticks from 0 to max_value with a step of 2
xticks = 0:1:max_value

# Create a histogram
histogram(dcj_dist_list, title="Distribution", label="", xlabel="DCJ Distance", ylabel="Frequency",  xlims=(0, max_value), xticks=(xticks, xticks))



In [ ]:
n = 3
x = 3
println(n, " gene(s), ", x, " duplicate(s)")
src, target = generate_genomes_with_xdup(n, x)

num_maps = 50

dcj_dist_list = randommap(src, target, num_maps, "info")

### visualization 

max_value = length(target)+2
println("\n**\nmax dcj distance = ", max_value, "\n")

# SOCS of distribution
# shape -- skewed right usually 
# outliers - none 
# center - depends 
# spread -- narrow 
min = minimum(dcj_dist_list)
max = maximum(dcj_dist_list)
avg = mean(dcj_dist_list)

println("min=", min)
println("max=", max)
println("avg=", avg, "\n")
println("range=", max-min)


# range for x-ticks from 0 to max_value with a step of 2
xticks = 0:1:max_value

# Create a histogram
histogram(dcj_dist_list, title="Distribution", label="", xlabel="DCJ Distance", ylabel="Frequency",  xlims=(0, max_value), xticks=(xticks, xticks))



In [ ]:
n = 4
x = 3
println(n, " gene(s), ", x, " duplicate(s)")
src, target = generate_genomes_with_xdup(n, x)

num_maps = 50

dcj_dist_list = randommap(src, target, num_maps, "info")

### visualization 

max_value = length(target)+2
println("\n**\nmax dcj distance = ", max_value, "\n")

# SOCS of distribution
# shape -- skewed right usually 
# outliers - none 
# center - depends 
# spread -- narrow 
min = minimum(dcj_dist_list)
max = maximum(dcj_dist_list)
avg = mean(dcj_dist_list)

println("min=", min)
println("max=", max)
println("avg=", avg, "\n")
println("range=", max-min)


# range for x-ticks from 0 to max_value with a step of 2
xticks = 0:1:max_value

# Create a histogram
histogram(dcj_dist_list, title="Distribution", label="", xlabel="DCJ Distance", ylabel="Frequency",  xlims=(0, max_value), xticks=(xticks, xticks))



In [ ]:
n = 4
x = 4
println(n, " gene(s), ", x, " duplicate(s)")
src, target = generate_genomes_with_xdup(n, x)

num_maps = 50

dcj_dist_list, min_src, min_map, max_src, max_map = randommap(src, target, num_maps, "info")

### visualization 

max_value = length(target)+2
println("\n**\nmax dcj distance = ", max_value, "\n")

# SOCS of distribution
# shape -- skewed right usually 
# outliers - none 
# center - depends 
# spread -- narrow 

min = minimum(dcj_dist_list)
max = maximum(dcj_dist_list)
avg = mean(dcj_dist_list)

println("min=", min)
println("max=", max)
println("avg=", avg, "\n")
println("range=", max-min)


# range for x-ticks from 0 to max_value with a step of 2
xticks = 0:1:max_value

# Create a histogram
histogram(dcj_dist_list, title="Distribution", label="", xlabel="DCJ Distance", ylabel="Frequency",  xlims=(0, max_value), xticks=(xticks, xticks))



In [ ]:
n = 5
x = 2

repeats = 10 # for every n,x pair 
max_range = 0 




println(n, " gene(s), ", x, " duplicate(s)")
src, target = generate_genomes_with_xdup(n, x)

num_maps = 50

dcj_dist_list = randommap(src, target, num_maps, "info")

### visualization 

max_value = length(target)+2
println("\n**\nmax dcj distance = ", max_value, "\n")

# SOCS of distribution
# shape -- skewed right usually 
# outliers - none 
# center - depends 
# spread -- narrow 
min = minimum(dcj_dist_list)
max = maximum(dcj_dist_list)
avg = mean(dcj_dist_list)

println("min=", min)
println("max=", max)
println("avg=", avg, "\n")
println("range=", max-min)


# range for x-ticks from 0 to max_value with a step of 2
xticks = 0:1:max_value

# Create a histogram
histogram(dcj_dist_list, title="Distribution", label="", xlabel="DCJ Distance", ylabel="Frequency",  xlims=(0, max_value), xticks=(xticks, xticks))

